<a href="https://colab.research.google.com/github/redadmiral/python-for-journalists/blob/main/lessons/Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
#!pip install --upgrade flask
#!pip install geopandas

import pandas as pd
import geopy

# Mapping

In dieser Einheit lernen wir, wie wir einen Datensatz auf einer Karte darstellen. Das funktioniert natürlich nur mit Daten, die auch etwas mit Raum zu tun haben. 

Ein Beispiel dafür sind zum Beispiel Wahldaten oder Adressen. 

## Flächen- und Punktdaten

Wahlergebnisse und Adressdaten zeigen auch die grundlegenden Unterschiede zwischen verschiedenen arten von Geo-Daten im Journalismus. Adressen sind Punkte auf einer Karte und Wahldaten werden als Flächen dargestellt. 

Die einfachste Variante sind Punkt-Daten. Als Beispiel nutzen wir einen kleinen Auszug aus dem Handelsregister. Dieser liegt [zum Download auf github](https://example.com).

Die Daten stammen von [offeneregister.de](https://offeneregister.de) und wurden von mir schon einmal ein wenig vorgefiltert.

Zuerst lesen wir die csv-Datei mit pandas ein:

In [77]:
adressen = pd.read_csv("HandelsregisterNuernberg.csv")

Ein kurzer Blick in die Datei zeigt uns, dass der Import erfolgreich war. Damit wir nicht den gesamten Bildschirm in Beschlag nehmen, können wir die `head`-Methode verwenden.

In [78]:
adressen.head(5)

,id,company_number,current_status,jurisdiction_code,name,registered_address,retrieved_at,register_flag_AD,register_flag_CD,register_flag_DK,...,native_company_number,registered_office,registrar,register_art,register_nummer,former_registrar,register_flag_,register_flag_Note:,_registerNummerSuffix,register_flag_Status information
0,6728,D3310V_HRB9108,currently registered,de,ARNDT Verwaltungsgesellschaft mit beschränkter...,"Schildgasse 11, 90403 Nürnberg.",2018-11-01T19:32:43Z,1,1,1,...,Nürnberg HRB 9108,Nürnberg,Nürnberg,HRB,9108,NaN,NaN,NaN,NaN,NaN
1,6731,D3310V_HRB26266,currently registered,de,designpanel elements for innovative architectu...,"Dagobertstraße 15, 90431 Nürnberg.",2018-11-09T16:55:45Z,1,1,1,...,Nürnberg HRB 26266,Nürnberg,Nürnberg,HRB,26266,NaN,NaN,NaN,NaN,NaN
2,6732,D3310V_HRB31142,currently registered,de,Bellota DE UG (haftungsbeschränkt),"Spitalgasse 1, 90403 Nürnberg.",2018-11-09T16:59:51Z,1,1,1,...,Nürnberg HRB 31142,Nürnberg,Nürnberg,HRB,31142,NaN,NaN,NaN,NaN,NaN
3,6733,D3310V_HRB29875,currently registered,de,NWS Bahnservice GmbH,"Fraunhofer Straße 10 a, 90409 Nürnberg.",2018-11-09T16:58:36Z,1,1,1,...,Nürnberg HRB 29875,Nürnberg,Nürnberg,HRB,29875,NaN,NaN,NaN,NaN,NaN
4,6734,D3310V_HRA16172,currently registered,de,Baliu Trans GmbH & Co. KG,"Industriestraße 1c, 90592 Schwarzenbruck.",2018-11-09T16:43:09Z,1,1,1,...,Nürnberg HRA 16172,Schwarzenbruck,Nürnberg,HRA,16172,NaN,NaN,NaN,NaN,NaN


## Geocoding

Um die Adressen auf einer Karte darstellen zu können, müssen sie in Koordinaten umgewandelt werden. Koordinaten auf einer Karte bestehen aus zwei Zahlen, die angeben wie weit nach rechts (Latitude) und wie weit nach oben (Longitude) man vom Mittelpunkt der Karte aus gehen muss. 

Für das eigentliche Geocoding verwenden wir eine Funktion aus geopandas. Diese Funktion kann für uns auf verschiedene APIs zugreifen, von denen Sie die Koordinaten für die Punkte beziehen kann.

Wir nutzen die API von OpenStreetMap "Nominatim". Wenn die Ergebnisse nicht gut sind, dann können wir auf eine andere API zugreifen, wie etwa Bing-Maps. Verschiedene Dienstleister sind für verschiedene Teile der Welt besser oder schlechter geeignet.

In [79]:
from geopandas.tools import geocode

Die Nominatim-API verlang von uns, dass wir einen User-Agent festlegen. In diesem identifizieren wir uns gegenüber der API, so dass die OSM-Macher\*innen uns kontaktieren können, wenn wir zu viele Anfragen an ihre Schnittstelle senden. 

In [80]:
geopy.geocoders.options.default_user_agent = "Just learning how to geocode data. Please contact me under lehner@post-box.me"

Nachdem wir den User-Agent gesetzt haben, können wir mit dem Geocoding loslegen:

In [81]:
geo = geocode(adressen["registered_address"], provider='nominatim')

Ein Blick in den neu entstandenen dataframe zeigt uns, dass geopandas aus den Adressen die Koordinaten extrahiert hat.

---



In [76]:
geo.head(5)

,geometry,address
0,POINT (11.07828 49.45721),"11, 13, Schildgasse, Altstadt, St. Sebald, Seb..."
1,POINT (11.01947 49.45618),"axis-expotechnik, 15, Dagobertstraße, Leyh, Nü..."
2,POINT (11.07845 49.45338),"Sultan, 1, Spitalgasse, Altstadt, St. Sebald, ..."
3,POINT (11.09786 49.46871),"10a, Fraunhoferstraße, Rennweg, Schoppershof, ..."
4,POINT (11.26293 49.35130),"Industriestraße, Ochenbruck, Schwarzenbruck, L..."


Jetzt importieren wir die entstandene Datei in Datawrapper. Datawrapper verlangt von uns mindestens drei Spalten:

1. Adresse
2. Latitude
3. Longitude

Also erstellen wir einen DataFrame mit diesen Spalten, indem wir zuerst dem `adressen`-Dataframe die Koordinaten in je einer eigenen Spalte hinzufügen. Auf die einzelnen Koordinaten im `POINT`-Objekt können wir über die `x` und `y` Parameter zugreifen:

In [ ]:
adressen["LAT"] = geo["geometry"].y
adressen["LONG"] = geo["geometry"].x

result = adressen[['id', 'name', 'registered_address', "LAT", "LONG"]]

Zum Schluss speichern wir die Datei nun als CSV-Datei ab, so dass wir sie in Datawrapper importieren können:

In [ ]:
result.to_csv("dwinput.csv")